# Indexing Dokumen Pariwisata (DetikTravel & KompasTravel)

Notebook ini melakukan tahap **indexing** untuk tugas Penelusuran Informasi:

1. Load hasil preprocessing (`corpus_clean.csv`)
2. Membuat `doc_id` dan menghitung panjang dokumen
3. Membuat **inverted index** (term → {doc_id: tf})
4. Menghitung statistik dasar: jumlah dokumen, ukuran vocabulary, df, idf
5. Menyimpan metadata dokumen & inverted index agar bisa dipakai di tahap TF-IDF / BM25 / pencarian query.


In [16]:
import pandas as pd
from collections import defaultdict
import math
import json
import os

# Sesuaikan dengan lokasi file kamu
# Kalau preprocessing.ipynb tadi menyimpan ke "data/corpus_clean.csv", pakai ini:
CORPUS_PATH = "data/corpus_clean.csv"

# Kalau file sekarang masih sejajar notebook:
# CORPUS_PATH = "corpus_clean.csv"

DOC_META_PATH = "data/doc_meta.csv"
INVERTED_INDEX_PATH = "data/inverted_index.json"

os.makedirs("data", exist_ok=True)

print("CORPUS_PATH:", CORPUS_PATH)


CORPUS_PATH: data/corpus_clean.csv


## 1. Load Corpus Bersih (corpus_clean.csv)

Kita baca hasil preprocessing yang berisi kolom:

- `url`
- `title`
- `word_count_raw`
- `word_count_clean`
- `content_raw`
- `content_clean`

Indexing akan menggunakan **content_clean** sebagai teks yang sudah diprepro.


In [17]:
df = pd.read_csv(CORPUS_PATH)

print("Jumlah dokumen:", len(df))
print(df.columns)

# Tampilkan 3 dokumen teratas untuk memastikan format sudah sesuai
df[["url", "title", "word_count_clean"]].head(3)


Jumlah dokumen: 4700
Index(['url', 'title', 'word_count_raw', 'word_count_clean', 'content_raw',
       'content_clean'],
      dtype='object')


,url,title,word_count_clean
0,https://travel.kompas.com/read/2025/11/11/1900...,Unik! Kamar Hotel Bertema Kereta Api di Jepang...,581
1,https://travel.kompas.com/read/2025/11/11/1925...,"4 Rekomendasi Wisata di Banyuwangi, Cocok untu...",356
2,https://travel.kompas.com/read/2025/05/03/1333...,"Itinerary Seharian di Bromo Jawa Timur, dari S...",535


## 2. Tambah `doc_id`, tokenisasi, dan panjang dokumen

- `doc_id` akan menjadi ID integer untuk setiap dokumen (0, 1, 2, ...).
- Token diambil dari `content_clean` yang sudah diproses.
- `doc_len` = jumlah token per dokumen (dipakai untuk statistik & BM25 nanti).


In [18]:
# Tambahkan kolom doc_id
df["doc_id"] = df.index.astype(int)

# Fungsi tokenisasi sederhana (content_clean sudah lower + bersih)
def tokenize(text):
    if pd.isna(text):
        return []
    return [t for t in str(text).split() if t]

df["tokens"] = df["content_clean"].apply(tokenize)
df["doc_len"] = df["tokens"].apply(len)

print("Jumlah dokumen:", len(df))
print(df[["doc_id", "doc_len"]].head())
print(df["doc_len"].describe())


Jumlah dokumen: 4700
   doc_id  doc_len
0       0      581
1       1      356
2       2      535
3       3      486
4       4      375
count    4700.000000
mean      401.006809
std       154.067986
min        64.000000
25%       309.000000
50%       381.000000
75%       453.000000
max      2247.000000
Name: doc_len, dtype: float64


## 3. Membangun Inverted Index

Struktur inverted index:

  - `inverted_index` : `dict[str, dict[int, int]]`  
  - key   : term (kata)
  - value : dictionary `{doc_id: tf}`


In [19]:

inverted_index = defaultdict(dict)

for _, row in df.iterrows():
    doc_id = int(row["doc_id"])
    tokens = row["tokens"]
    
    # Hitung term frequency di 1 dokumen
    term_counts = defaultdict(int)
    for tok in tokens:
        term_counts[tok] += 1
    
    # Masukkan ke inverted index
    for term, tf in term_counts.items():
        inverted_index[term][doc_id] = tf

vocab_size = len(inverted_index)
print("Jumlah term unik (vocabulary size):", vocab_size)


Jumlah term unik (vocabulary size): 37662


## 4. Hitung Document Frequency (df) dan Inverse Document Frequency (idf)

- `N`  = total dokumen
- `df` = jumlah dokumen yang mengandung term tersebut
- `idf` (smoothed) kita pakai rumus:

\[
idf(t) = \log\left(\frac{N + 1}{df(t) + 1}\right) + 1
\]

Rumus ini sering dipakai sebagai varian smoothed IDF (menghindari pembagian 0).


In [20]:
N = len(df)

df_term = {term: len(postings) for term, postings in inverted_index.items()}
idf = {term: math.log((N + 1) / (df_t + 1)) + 1 for term, df_t in df_term.items()}

print("Total dokumen (N):", N)
print("Contoh 5 term pertama (term, df, idf):")
for i, (term, df_t) in enumerate(df_term.items()):
    if i >= 5:
        break
    print(f"- {term!r}: df={df_t}, idf={idf[term]:.4f}")


Total dokumen (N): 4700
Contoh 5 term pertama (term, df, idf):
- 'unik': df=581, idf=3.0891
- 'kamar': df=373, idf=3.5313
- 'hotel': df=872, idf=2.6836
- 'tema': df=224, idf=4.0394
- 'kereta': df=725, idf=2.8680


## 5. Simpan Metadata Dokumen & Inverted Index

Untuk memudahkan pemakaian di notebook lain (misalnya: TF-IDF, BM25, search UI), kita simpan:

1. **Metadata dokumen**:
   - `doc_id`
   - `url`
   - `title`
   - `doc_len`

2. **Inverted index** ke file JSON:
   - Format: `term -> list of {"doc_id": int, "tf": int}`  
     (supaya aman untuk di-JSON-kan, karena key dictionary tidak boleh integer).


In [21]:
# 1) Simpan metadata dokumen
doc_meta = df[["doc_id", "url", "title", "doc_len"]].copy()
doc_meta.to_csv(DOC_META_PATH, index=False, encoding="utf-8")

print("Doc metadata disimpan ke:", DOC_META_PATH)
print(doc_meta.head(3))

# 2) Siapkan inverted_index dalam format serializable
serializable_index = {}
for term, postings in inverted_index.items():
    serializable_index[term] = [
        {"doc_id": int(doc_id), "tf": int(tf)}
        for doc_id, tf in postings.items()
    ]

with open(INVERTED_INDEX_PATH, "w", encoding="utf-8") as f:
    json.dump(serializable_index, f, ensure_ascii=False)

print("Inverted index disimpan ke:", INVERTED_INDEX_PATH)
print("Ukuran vocabulary:", len(serializable_index))


Doc metadata disimpan ke: data/doc_meta.csv
   doc_id                                                url  \
0       0  https://travel.kompas.com/read/2025/11/11/1900...   
1       1  https://travel.kompas.com/read/2025/11/11/1925...   
2       2  https://travel.kompas.com/read/2025/05/03/1333...   

                                               title  doc_len  
0  Unik! Kamar Hotel Bertema Kereta Api di Jepang...      581  
1  4 Rekomendasi Wisata di Banyuwangi, Cocok untu...      356  
2  Itinerary Seharian di Bromo Jawa Timur, dari S...      535  
Inverted index disimpan ke: data/inverted_index.json
Ukuran vocabulary: 37662
Inverted index disimpan ke: data/inverted_index.json
Ukuran vocabulary: 37662


## 6. Utility: Cek Posting List untuk Satu Term

Fungsi kecil untuk mengecek apakah indexing sudah masuk akal:

- Input: sebuah kata (misalnya `"pantai"`)
- Output: daftar beberapa dokumen yang mengandung kata itu, beserta `tf` dan judul.


In [22]:
def lihat_posting(term, top_n=10):
    term = term.lower().strip()
    if term not in inverted_index:
        print(f"Term {term!r} tidak ditemukan di index.")
        return
    
    postings = inverted_index[term]
    print(f"Term {term!r} muncul di {len(postings)} dokumen.")
    
    # Urutkan berdasarkan tf tertinggi
    sorted_postings = sorted(postings.items(), key=lambda x: -x[1])[:top_n]
    
    for doc_id, tf in sorted_postings:
        row = df.loc[df["doc_id"] == doc_id].iloc[0]
        title = str(row["title"])
        print(f"- doc_id={doc_id}, tf={tf}, title={title[:80]}...")

# Contoh pemakaian:
lihat_posting("pantai", top_n=5)


Term 'pantai' muncul di 893 dokumen.
- doc_id=2776, tf=46, title=18 Wisata Pantai Gunungkidul yang Paling Terkenal, Cocok untuk Liburan Sekolah...
- doc_id=4619, tf=46, title=18 Wisata Pantai Gunungkidul yang Paling Terkenal, Cocok untuk Liburan Sekolah...
- doc_id=1866, tf=39, title=15 Tempat Wisata Bali, Cocok untuk Libur Panjang...
- doc_id=3808, tf=39, title=15 Tempat Wisata Bali, Cocok untuk Libur Panjang...
- doc_id=1858, tf=36, title=10 Tempat Wisata Pantai Tersembunyi di Bali, Indah dan Sepi...


## 7. Ringkasan

Pada tahap indexing ini kita sudah:

- ✅ Menggunakan `corpus_clean.csv` hasil preprocessing
- ✅ Menambahkan `doc_id`, token, dan panjang dokumen
- ✅ Membangun **inverted index**: term → {doc_id: tf}
- ✅ Menghitung df dan idf untuk setiap term
- ✅ Menyimpan:
  - `data/doc_meta.csv`
  - `data/inverted_index.json`

Tahap berikutnya dalam tugas IR:

1. Menggunakan index + idf untuk membangun model **TF-IDF** dan **BM25**
2. Membuat fungsi pencarian (input query → daftar dokumen ter-relevan)
3. Melakukan evaluasi (Precision, Recall, F1, MAP) untuk beberapa query uji.
